# HPH第二版实验

### 思路

*id统计相关*
1. item_id, shop_id的交易量, 交易率
2. item_id, shop_id7天是否新出现, 3天是否新出现, 昨天是否新出现


In [26]:
import sys
sys.path.append("..")
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime

%matplotlib inline
offline_df = pd.read_table('../../round1_ijcai_18_train_20180301.txt', sep=' ')
# online_df = pd.read_table('../../round1_ijcai_18_test_a_20180301.txt', sep=' ')

# 1 数据集统一处理

In [ ]:
# 时间处理: 分离天, 星期几, 上中下午/晚上, 小时数

from datetime import datetime

def extract_date(x):
    d = datetime.fromtimestamp(x)
    return d.strftime('%Y-%m-%d')
def extract_weekday(x):
    d = datetime.fromtimestamp(x)
    return d.weekday()
def extract_hour(x):
    d = datetime.fromtimestamp(x)
    return d.hour

offline_df['date'] = offline_df['context_timestamp'].apply(lambda x: extract_date(x))
offline_df['weekday'] = offline_df['context_timestamp'].apply(lambda x: extract_weekday(x))
offline_df['hour'] = offline_df['context_timestamp'].apply(lambda x: extract_hour(x))

In [ ]:
%run ../util/time_utils.py

offline_df = getColDupByDate(offline_df, 'date', 'item_id', 3)
offline_df = getColDupByDate(offline_df, 'date', 'item_brand_id', 3)
offline_df = getColDupByDate(offline_df, 'date', 'shop_id', 3)
offline_df = getColDupByDate(offline_df, 'date', 'user_id', 3)

offline_df = getColDupByDate(offline_df, 'date', 'item_id', 7)
offline_df = getColDupByDate(offline_df, 'date', 'item_brand_id', 7)
offline_df = getColDupByDate(offline_df, 'date', 'shop_id', 7)
offline_df = getColDupByDate(offline_df, 'date', 'user_id', 7)
offline_df = getColDupByDate(offline_df, 'date', 'item_id', 1)
offline_df = getColDupByDate(offline_df, 'date', 'item_brand_id', 1)
offline_df = getColDupByDate(offline_df, 'date', 'shop_id', 1)
offline_df = getColDupByDate(offline_df, 'date', 'user_id', 1)


In [ ]:
# 统计信息附加

def getColTradeRate(df, idCol):
    rateCol = idCol + '_tr'
    pvCol = idCol + '_pv'
    try:
        del df[rateCol]
        del df[pvCol]
    except:
        pass
    a = offline_df.groupby([idCol]).agg({'is_trade':'sum'})
    b = offline_df.groupby([idCol]).agg({'is_trade':'size'})
    c = a.join(b, lsuffix="_sum", rsuffix="_size")
    c[rateCol] = c['is_trade_sum'] / c['is_trade_size']
    c[pvCol] = c['is_trade_size']
    return offline_df.join(c[[rateCol, pvCol]], on=idCol)
    

# 各类id的 交易量, 交易率
offline_df = getColTradeRate(offline_df, 'item_city_id')
offline_df = getColTradeRate(offline_df, 'item_id')
offline_df = getColTradeRate(offline_df, 'shop_id')
offline_df = getColTradeRate(offline_df, 'item_brand_id')
offline_df = getColTradeRate(offline_df, 'user_occupation_id')


In [ ]:
# 分桶
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

offline_df['item_city_id_pv_cut'] = pd.cut(offline_df['item_city_id_pv'].values, bins=10, retbins=False)
offline_df['item_id_pv_cut'] = pd.cut(offline_df['item_id_pv'].values, bins=10, retbins=False)
offline_df['shop_id_pv_cut'] = pd.cut(offline_df['shop_id_pv'].values, bins=10, retbins=False)
offline_df['item_brand_id_pv_cut'] = pd.cut(offline_df['item_brand_id_pv'].values, bins=10, retbins=False)
offline_df['user_occupation_id_pv_cut'] = pd.cut(offline_df['user_occupation_id_pv'].values, bins=10, retbins=False)

offline_df['item_city_id_pv_cut'] = le.fit_transform(offline_df['item_city_id_pv_cut'].copy())
offline_df['item_id_pv_cut'] = le.fit_transform(offline_df['item_id_pv_cut'].copy())
offline_df['shop_id_pv_cut'] = le.fit_transform(offline_df['shop_id_pv_cut'].copy())
offline_df['item_brand_id_pv_cut'] = le.fit_transform(offline_df['item_brand_id_pv_cut'].copy())
offline_df['user_occupation_id_pv_cut'] = le.fit_transform(offline_df['user_occupation_id_pv_cut'].copy())

In [ ]:
# standardization and scaling

from sklearn import preprocessing

def scale(df, idCol):
    min_max_scaler = preprocessing.MinMaxScaler()
    a = min_max_scaler.fit_transform(df[idCol].values.reshape(-1, 1))
    return pd.Series(a.reshape(1, -1)[0])

# 令数据再少一些?
offline_df['shop_score_service_scaled'] = scale(offline_df, 'shop_score_service')
offline_df['shop_score_delivery_scaled'] = scale(offline_df, 'shop_score_delivery')
offline_df['shop_score_description_scaled'] = scale(offline_df, 'shop_score_description')
offline_df['shop_review_positive_rate_scaled'] = scale(offline_df, 'shop_review_positive_rate')
offline_df['item_city_id_tr_scaled'] = scale(offline_df, 'item_city_id_tr')
offline_df['item_id_tr_scaled'] = scale(offline_df, 'item_id_tr')
offline_df['shop_id_tr_scaled'] = scale(offline_df, 'shop_id_tr')
offline_df['item_brand_id_tr_scaled'] = scale(offline_df, 'item_brand_id_tr')
offline_df['user_occupation_id_scaled'] = scale(offline_df, 'user_occupation_id_tr')

# offline_df = offline_df.dropna()

# 2 生成矩阵数据

In [7]:
used = 78261
D = offline_df['context_timestamp']
X = offline_df[
    [
        'item_id', 'item_brand_id', 'item_city_id', 'user_id', 'context_page_id', 'context_id', 'shop_id', 
        'item_id_dup_g_7', 'shop_id_dup_g_7', 'item_brand_id_dup_g_7', 'user_id_dup_g_7',
#         'item_id_dup_g_1', 'shop_id_dup_g_1', 'item_brand_id_dup_g_1', 'user_id_dup_g_1',
#         'item_id_dup_g_3', 'shop_id_dup_g_3', 'item_brand_id_dup_g_3', 'user_id_dup_g_3',
        'item_id_pv_cut', 'shop_id_pv_cut', 'user_occupation_id_pv_cut', 
         'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 
        'user_gender_id', 'weekday', 'hour', 'user_age_level', 'user_star_level', 
       'user_occupation_id', 
       'item_city_id_tr', 'item_city_id_pv_cut', 'item_id_tr', 'item_id_pv',
       'shop_id_tr', 'shop_id_pv', 'item_brand_id_tr', 'item_brand_id_pv',
       'user_occupation_id_pv', 'shop_score_delivery',
        'shop_score_description', 'shop_review_positive_rate', 'shop_score_service'      
    ]
].values
#        'shop_score_delivery_scaled', 'shop_score_description_scaled',
#        'shop_review_positive_rate_scaled', 'item_city_id_tr_scaled',
#        'item_id_tr_scaled', 'shop_id_tr_scaled', 'item_brand_id_tr_scaled',k
#        'user_occupation_id_scaled', 'shop_score_service_scaled', 
        
y = offline_df[['is_trade']].values.ravel()

# 3 GridSearchCV 超参搜索

In [9]:
from sklearn.model_selection import GridSearchCV as GSCV   #Perforing grid search
from sklearn.metrics import make_scorer
from sklearn.metrics import log_loss
import xgboost

% run ../util/time_series_split.py


dtsv = DateTimeSplit(dateSeries=D, fmt="%Y-%m-%d")
score_func = make_scorer(log_loss, greater_is_better=False, needs_proba=True)

# for train_i, test_i in dtsv.split(X, y):
#     X_train, y_train = X[train_i], y[test_i]
#     X_test, y_test = X[test_i], y[test_i]
    
#     print(X_train.shape, y_train.shape)
#     print(X_test.shape, y_test.shape)

parameters = {
  'nthread':[4], #when use hyperthread, xgboost may become slower
  'learning_rate': [0.05], #so called `eta` value
  'max_depth': [6],
  'n_estimators': [5]
}

xgb = xgboost.XGBClassifier()
gs = GSCV(xgb, parameters, scoring=score_func, cv=dtsv)
gs.fit(X, y)

    
# best_parameters, score, _ = max(gs.cv_results_, key=lambda x: x[1])
    

GridSearchCV(cv=<__main__.DateTimeSplit object at 0x1a20fa4860>,
       error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'nthread': [4], 'learning_rate': [0.05], 'max_depth': [6], 'n_estimators': [5]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=make_scorer(log_loss, greater_is_better=False, needs_proba=True),
       verbose=0)

In [10]:
# best_parameters, score = max(gs.cv_results_, key=lambda x: x[1])
print(gs.cv_results_)

{'mean_fit_time': array([ 1.30268296]), 'std_fit_time': array([ 0.4735773]), 'mean_score_time': array([ 0.04144303]), 'std_score_time': array([ 0.00406109]), 'param_learning_rate': masked_array(data = [0.05],
             mask = [False],
       fill_value = ?)
, 'param_max_depth': masked_array(data = [6],
             mask = [False],
       fill_value = ?)
, 'param_n_estimators': masked_array(data = [5],
             mask = [False],
       fill_value = ?)
, 'param_nthread': masked_array(data = [4],
             mask = [False],
       fill_value = ?)
, 'params': [{'learning_rate': 0.05, 'max_depth': 6, 'n_estimators': 5, 'nthread': 4}], 'split0_test_score': array([-0.50556711]), 'split1_test_score': array([-0.50549813]), 'split2_test_score': array([-0.50536553]), 'split3_test_score': array([-0.50496089]), 'split4_test_score': array([-0.50418305]), 'split5_test_score': array([-0.50382902]), 'mean_test_score': array([-0.50494607]), 'std_test_score': array([ 0.00065093]), 'rank_test_score'

# 4 kfold 测试

In [23]:
X = offline_df[
    [
        'date',
        'item_id', 'item_brand_id', 'item_city_id', 'user_id', 'context_page_id', 'context_id', 'shop_id', 
        'item_id_dup_g_7', 'shop_id_dup_g_7', 'item_brand_id_dup_g_7', 'user_id_dup_g_7',
#         'item_id_dup_g_1', 'shop_id_dup_g_1', 'item_brand_id_dup_g_1', 'user_id_dup_g_1',
#         'item_id_dup_g_3', 'shop_id_dup_g_3', 'item_brand_id_dup_g_3', 'user_id_dup_g_3',
        'item_id_pv_cut', 'shop_id_pv_cut', 'user_occupation_id_pv_cut', 
         'item_price_level', 'item_sales_level', 'item_collected_level', 'item_pv_level', 
        'user_gender_id', 'weekday', 'hour', 'user_age_level', 'user_star_level', 
       'user_occupation_id', 
       'item_city_id_tr', 'item_city_id_pv_cut', 'item_id_tr', 'item_id_pv',
       'shop_id_tr', 'shop_id_pv', 'item_brand_id_tr', 'item_brand_id_pv',
       'user_occupation_id_pv', 'shop_score_delivery',
        'shop_score_description', 'shop_review_positive_rate', 'shop_score_service'      
    ]
]
#        'shop_score_delivery_scaled', 'shop_score_description_scaled',
#        'shop_review_positive_rate_scaled', 'item_city_id_tr_scaled',
#        'item_id_tr_scaled', 'shop_id_tr_scaled', 'item_brand_id_tr_scaled',k
#        'user_occupation_id_scaled', 'shop_score_service_scaled', 
        
y = offline_df[['is_trade']]

In [24]:
train_input = X
# train_input['is_trade'] = offline_df['is_trade']
from sklearn.model_selection import GridSearchCV   #Perforing grid search
from sklearn.metrics import make_scorer
from sklearn.metrics import log_loss
import xgboost
%run ../yuhua/BaseFrame.py


it = 0
def tatx(train_df, test_df, valid_df):

    X_train = train_df.copy()
    del X_train['is_trade']

    y_train = train_df['is_trade']
    
    X_test = test_df.copy()
    del X_test['is_trade']
    y_test = test_df['is_trade']
    
    X_valid = valid_df.copy()
    del X_valid['is_trade']
    y_valid = valid_df['is_trade']

       
    params = {
        'nthread': 20
    }
    m = xgboost.XGBClassifier(**params)
    m.fit(X_train, y_train)

    
    l_train = log_loss(y_train, m.predict_proba(X_train))
    l_test = log_loss(y_test, m.predict_proba(X_test))
    l_valid = log_loss(y_valid, m.predict_proba(X_valid))

    return l_train, l_test, l_valid

frame = BaseFrame(train_input, 0.05)
frame.kfold_by_date('date', tatx)
# 0.077189, 0.076092
# test = 0.077047  valid = 0.07597
# test = 0.077077  valid = 0.075944

2018-09-18 to 2018-09-18 is training set, 2018-09-19 is test set, 2018-09-24 is valid set, start training ... 


KeyError: 'is_trade'